# Part 2

In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/12500/824022/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1575169444&Signature=eVeXAucRcVmn6Ra14LLaGqhnIYCB1eHkVVgCuCq3PALafECs71RAZOt0Q323rNjpKFq%2BVn62jsi9qzJqTVkFyZerXLoi7cCbvdlCiDA39Er84Dd8ckj549ncSE3PLqDIg%2F%2F4Mc5GOCM%2B09sHddQDllUb8IE2tNzvR6cw7WWO94tliwrw6S9adDpdSN%2BxnCYBPLKSK%2FRNV9kZXbQkPEoc%2Fed9rd61msFIY%2B4mb74ot6r07XSIj%2FaJ2Bgiuiduc94BHbviGWC3ISMVeKOPYGqco1x2lTjY8c5rbrMgyMN2CuEzIBsXV7Ya7XqJbfGwXbPVSlmsZs7O3aYeYNzlu7UtGg%3D%3D" -O "train.csv.zip" -c

In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/12500/824022/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1575169411&Signature=jtOtKUkUKARCIaIjgVMYX1dQTuIvYqqCjcL2mNVVw%2BvhOzmeavb%2FTCYYT7X52d2Ph3ssh6YUEqY%2BQuGuI0tnOXXSA177RaaF9DBTXoFoKXiznFUx0UGwI0B2rdKim%2BKJC0JhxEyiVbMu28wlLoU59P0wQ4xK6sL0ZIKgc00%2Bmca7sWTcCgAaLHb9k2CUa8HZdPKX2fvZDdEtz2CFy0klqUK2lyc4dPCZCgGTSUVYLkR%2BWIh%2F%2FRvN9vNWEr6zmrhuGG3tplDNy8fjGxLIPcNKjZfOWJiXw4Kqoy7dJgUxscIyi0ySBJKIDGpCuHbOJpjYrvVOZzrIjMCZcR4zOj9NbQ%3D%3D" -O "test.csv.zip" -c

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.model_selection import train_test_split
import string
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem.snowball import SnowballStemmer
from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout,CuDNNLSTM,Flatten,Layer,GlobalAveragePooling1D,add,Conv1D
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from kerafrom google.colab import files
import os.callbacks import EarlyStopping
import tensorflow as tf
from google.colab import files
import os
from sklearn import metrics
from keras import backend as K

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
Using TensorFlow backend.


In [5]:
# Reading data
df_train = pd.read_csv('train.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
df_test = pd.read_csv('test.csv.zip', compression='zip', header=0, sep=',', quotechar='"')

print("shape of train data:", df_train.shape)

shape of train data: (1804874, 45)


In [0]:
# Assigning class label based on target variable
def label(target):                           
    return 0 if target < 0.5 else 1
        
df_train['class'] = df_train.apply(lambda x: label(x['target']), axis= 1)  

# Splitting data into Train-CV

In [7]:
X_train, X_cv, y_train, y_cv = train_test_split(df_train, df_train['class'], test_size=0.1)
print('Data points in train {} and cv {}'. format(X_train.shape[0],X_cv.shape[0]))

X_train1 = X_train['comment_text'].astype(str)
X_cv1 = X_cv['comment_text'].astype(str)
X_test = df_test['comment_text'].astype(str)

Data points in train 1624386 and cv 180488


In [0]:
num_words = 35000 # using top 35000 frequent words
max_len = 100 
emb_size = 128
#Tokenizer converts words to number based their frequency of occurance.
t = Tokenizer(num_words = num_words)
t.fit_on_texts(list(X_train1))
# converting each comment to sequence
X_train1 = t.texts_to_sequences(X_train1)
X_cv1=t.texts_to_sequences(X_cv1)
test = t.texts_to_sequences(X_test)

# Padding with max lenght of 100
X_train1 = sequence.pad_sequences(X_train1, maxlen=max_len)
X_cv1 = sequence.pad_sequences(X_cv1, maxlen=max_len)
X_test = sequence.pad_sequences(test, maxlen=max_len)


In [0]:
# function for Standard auc
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

# Custom AUC for unintended bias in words

In [0]:
# code for calculating Custom auc taken from kaggle
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive
TOXICITY_COLUMN = 'target'

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [0]:
# overall auc adding bias to each subgroup
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)

In [0]:
# converting label and identity colums to boolean values
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df
X_cv = convert_dataframe_to_bool(X_cv)

In [0]:
train_y = to_categorical(y_train)
cv_y = to_categorical(y_cv)

# MODEL 1

In [0]:
#https://realpython.com/python-keras-text-classification/
inp = Input(shape = (max_len, ))
layer = Embedding(num_words, emb_size)(inp)
layer = Bidirectional(LSTM(50, return_sequences=True, recurrent_dropout=0.15))(layer)
layer = GlobalMaxPool1D()(layer)
layer = Dense(50, activation='relu')(layer)
layer = Dropout(0.2)(layer)
layer = Dense(2, activation='sigmoid')(layer)

model = Model(inputs=inp, outputs=layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
model.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 100)               0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 100, 128)          4480000   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 100, 100)          71600     
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 100)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_11 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 102

In [0]:
history = model.fit(X_train1, train_y, batch_size=1024, epochs=5, validation_data=(X_cv1,cv_y),verbose=1)

Train on 1624386 samples, validate on 180488 samples
Epoch 1/5
1624386/1624386 [==============================] - 512s 315us/step - loss: 0.1281 - auc: 0.9849 - val_loss: 0.1322 - val_auc: 0.9862
Epoch 2/5
1624386/1624386 [==============================] - 513s 316us/step - loss: 0.1183 - auc: 0.9872 - val_loss: 0.1323 - val_auc: 0.9879
Epoch 3/5
1624386/1624386 [==============================] - 511s 314us/step - loss: 0.1065 - auc: 0.9886 - val_loss: 0.1396 - val_auc: 0.9891
Epoch 4/5
1624386/1624386 [==============================] - 510s 314us/step - loss: 0.0920 - auc: 0.9897 - val_loss: 0.1544 - val_auc: 0.9901
Epoch 5/5
1624386/1624386 [==============================] - 521s 321us/step - loss: 0.0761 - auc: 0.9907 - val_loss: 0.1868 - val_auc: 0.9911


In [0]:
TOXICITY_COLUMN = 'target'
MODEL_NAME = 'model1'
X_cv[MODEL_NAME] = model.predict(X_cv1)[:,1]
bias_metrics_df = compute_bias_metrics_for_model(X_cv, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
bias_metrics_df

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
6,black,1436,0.737179,0.775611,0.917753
7,white,2475,0.764191,0.778985,0.928894
2,homosexual_gay_or_lesbian,1083,0.767462,0.787029,0.924113
5,muslim,2076,0.785124,0.809262,0.924849
4,jewish,801,0.792037,0.852692,0.888648
1,female,5382,0.859447,0.880362,0.909955
3,christian,4134,0.859748,0.907749,0.880346
0,male,4504,0.860599,0.868936,0.921170
8,psychiatric_or_mental_illness,489,0.873470,0.846856,0.935578


In [0]:
get_final_metric(bias_metrics_df, calculate_overall_auc(X_cv, MODEL_NAME))

0.8658840551094565

# Observation:
1.We can see stardard AUC on CV data is 0.99 which is due to class imbalance .

2.Due to adding weights to each identity columns ,the overall weighted average AUC is less as compared to standard AUC.

3.From metric table above jewish subgroup has comparetively less AUC score.

# MODEL 2

In [0]:
#https://realpython.com/python-keras-text-classification/
inp = Input(shape = (max_len, ))
layer = Embedding(num_words, emb_size)(inp)
layer = Bidirectional(LSTM(50,return_sequences=True,  recurrent_dropout=0.15))(layer)
layer = Bidirectional(LSTM(50,return_sequences=True ))(layer)
layer = GlobalMaxPool1D()(layer)
layer = Dense(64, activation='relu')(layer)
layer = Dropout(0.2)(layer)
layer = Dense(32, activation='relu')(layer)
layer = Dense(2, activation='sigmoid')(layer)

model2 = Model(inputs=inp, outputs=layer)
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
model2.summary()

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 100)               0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 100, 128)          4480000   
_________________________________________________________________
bidirectional_9 (Bidirection (None, 100, 100)          71600     
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 100, 100)          60400     
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 100)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 64)                6464      
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0  

In [0]:

history = model2.fit(X_train1, train_y, batch_size=1024, epochs=3, validation_data=(X_cv1,cv_y),verbose=1)

Train on 1624386 samples, validate on 180488 samples
Epoch 1/3
1624386/1624386 [==============================] - 974s 600us/step - loss: 0.1582 - auc: 0.9678 - val_loss: 0.1309 - val_auc: 0.9840
Epoch 2/3
1624386/1624386 [==============================] - 959s 590us/step - loss: 0.1252 - auc: 0.9861 - val_loss: 0.1291 - val_auc: 0.9872
Epoch 3/3
1624386/1624386 [==============================] - 956s 588us/step - loss: 0.1149 - auc: 0.9881 - val_loss: 0.1314 - val_auc: 0.9887


In [0]:
MODEL_NAME = 'model2'
X_cv[MODEL_NAME] = model2.predict(X_cv1)[:,1]
bias_metrics_df = compute_bias_metrics_for_model(X_cv, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
bias_metrics_df

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
6,black,1436,0.777643,0.777804,0.964324
2,homosexual_gay_or_lesbian,1083,0.795574,0.804998,0.960011
7,white,2475,0.810180,0.797744,0.964887
5,muslim,2076,0.828485,0.841605,0.956264
4,jewish,801,0.863732,0.866221,0.954987
8,psychiatric_or_mental_illness,489,0.890720,0.873463,0.959006
1,female,5382,0.899197,0.906251,0.949764
0,male,4504,0.899653,0.892357,0.958189
3,christian,4134,0.906246,0.927169,0.935674


In [0]:
get_final_metric(bias_metrics_df, calculate_overall_auc(X_cv, MODEL_NAME))

0.8988726969000456

# Observation:

1.Adding another bi-lstm layer and slightly tweeking model1 architecture has sifincantly improved overall AUC score in model 2.

# MODEL3

In [0]:
MAX_SEQUENCE_LENGTH = 150 
MAX_NUM_WORDS = 200000 # changed max frequent words to 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2


In [9]:
# loading word vectors
embeddings_index = {}
with open('/content/drive/My Drive/glove.6B.300d.txt', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
print('Preparing embedding matrix.')
X_train = df_train['comment_text'].astype(str)
X_test = df_test['comment_text'].astype(str)
#Tokenizer converts words to number based their frequency of occurance.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


Preparing embedding matrix.
Found 397708 unique tokens.



In [0]:
# converting word to sequencrs
X = tokenizer.texts_to_sequences(X_train)
test = tokenizer.texts_to_sequences(X_test)

In [0]:
# zero padding
X = sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH )
X_test = sequence.pad_sequences(test, maxlen=MAX_SEQUENCE_LENGTH )



In [0]:
def label(target):
    return 0 if target < 0.5 else 1
        
df_train['class'] = df_train.apply(lambda x: label(x['target']), axis= 1)

## Attention layer

In [0]:

# credits: https://gist.github.com/cbaziotis/7ef97ccf71cbc14366835198c09809d2
from keras import backend as K
from keras import initializers,regularizers,constraints
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]




## Giving weights for identity words 

In [0]:
# Idea taken from 2nd place solution
y_aux_train = df_train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']].values # subgroup columns
identity_columns = [
        'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
        'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

# calculating weights for each data points. This will be used during loss calculation.
# Each of the 4 sections can contribute 25% to the overall weight.

# Overall
weights = np.ones((len(df_train),)) / 4

# Subgroup
weights += (df_train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4

# Background Positive, Subgroup Negative
weights += (( (df_train['target'].values>=0.5).astype(bool).astype(np.int) +
   (df_train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4

# Background Negative, Subgroup Positive
weights += (( (df_train['target'].values<0.5).astype(bool).astype(np.int) +
   (df_train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
loss_weight = 1.0 / weights.mean()

In [30]:
inp = Input(shape=(150,))
x = Embedding(num_words,
                 EMBEDDING_DIM,
                weights=[embedding_matrix],
                input_length=MAX_SEQUENCE_LENGTH, # load pre-trained word embeddings into an Embedding layer
                trainable=False)(inp)            # note that we set trainable = False so as to keep the embeddings fixed

x = Bidirectional(LSTM(256,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))(x)
x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
at = AttentionWithContext()(x)
x = Dense(16, activation="relu")(at)
ide_output = Dense(6, activation="sigmoid")(x) # predicting subgroups
out = Dense(1, activation="sigmoid")(x)


model = Model(inputs=[inp], outputs=[out, ide_output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'],loss_weights=[loss_weight, 1.0] )
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 150, 300)     60000000    input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 150, 512)     1140736     embedding_3[0][0]                
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 150, 256)     657408      bidirectional_3[0][0]            
____________________________________________________________________________________________

In [0]:
history = model.fit(X,[ df_train['class'],y_aux_train], batch_size=730, epochs=4, validation_split=0.2)



Train on 1443899 samples, validate on 360975 samples
Epoch 1/4
1443899/1443899 [==============================] - 1267s 878us/step - loss: 0.6000 - dense_3_loss: 0.1498 - dense_2_loss: 0.1192 - dense_3_acc: 0.9446 - dense_2_acc: 0.8535 - val_loss: 0.5538 - val_dense_3_loss: 0.1375 - val_dense_2_loss: 0.1124 - val_dense_3_acc: 0.9463 - val_dense_2_acc: 0.8490
Epoch 2/4
1443899/1443899 [==============================] - 1246s 863us/step - loss: 0.5192 - dense_3_loss: 0.1284 - dense_2_loss: 0.1073 - dense_3_acc: 0.9505 - dense_2_acc: 0.8563 - val_loss: 0.5317 - val_dense_3_loss: 0.1315 - val_dense_2_loss: 0.1097 - val_dense_3_acc: 0.9481 - val_dense_2_acc: 0.8491
Epoch 3/4
1443899/1443899 [==============================] - 1245s 862us/step - loss: 0.4997 - dense_3_loss: 0.1228 - dense_2_loss: 0.1055 - dense_3_acc: 0.9522 - dense_2_acc: 0.8564 - val_loss: 0.5193 - val_dense_3_loss: 0.1280 - val_dense_2_loss: 0.1085 - val_dense_3_acc: 0.9496 - val_dense_2_acc: 0.8491
Epoch 4/4
1443899/144

1.Overall AUC on test data obtained was 0.92172

In [0]:
y_test = model.predict(X_test)
sub = pd.read_csv('/content/drive/My Drive/sample_submission.csv')
sub['prediction'] = y_test[0]
sub.to_csv('submission.csv', index=False)


 ## Procedure Followed:
1.Built three models with Bi directional Lstm with different architectures .

2.For first two models just trained them without any word embedding matrix and without any weights to background and subgroups.

3.Both first two models were not crossing the custom AUC score of 0.90 even while increasing the epochs.

4.Built another model with more complex aechitecture which includes adding weights to subgroups and backgrounds , adding word embedding matrix and with state of the art attention layer which helped to improve AUC 0.922


In [38]:
from prettytable import PrettyTable


conclusion= PrettyTable()
conclusion.field_names = [ "Model", "architecture", " custom cv AUC",'custom test AUC ']

conclusion.add_row(["MODEL 1", "1-Bi lstm  with no word embedding matrix", 0.8658,'--'])
conclusion.add_row(["MODEL 2", "2-Bi lstm  with no word embedding matrix", 0.8988, '--'])
conclusion.add_row(["MODEL 3", "2-Bi lstm  with word embedding matrix and attention layer",  '--',0.9217])
print(conclusion)

+---------+-----------------------------------------------------------+----------------+------------------+
|  Model  |                        architecture                       |  custom cv AUC | custom test AUC  |
+---------+-----------------------------------------------------------+----------------+------------------+
| MODEL 1 |          1-Bi lstm  with no word embedding matrix         |     0.8658     |        --        |
| MODEL 2 |          2-Bi lstm  with no word embedding matrix         |     0.8988     |        --        |
| MODEL 3 | 2-Bi lstm  with word embedding matrix and attention layer |       --       |      0.9217      |
+---------+-----------------------------------------------------------+----------------+------------------+


# Conclusion:
1.Model 1 and 2 failed to distinguish between the classes even though using bi- directional LSTM because it is unable to capture long term dependency of words there by misinterpreting identity words to be toxic.

2.By including attention layer in model3  helped to remember long term dependencies of words there by understanding context of sentences which increased AUC by significant margin.

3.Glove vectors of words and adding loss weights to each identity groups is key booster for AUC.
